<a href="https://colab.research.google.com/github/ortenburger/apoHackathon/blob/master/PandasNormalizedQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
#os.chdir("gdrive/My Drive/Hackathon")
!ls
store = pd.HDFStore('foo')

2013_Qualitätsberichte.zip  2016_Qualitätsberichte.zip	adresses2019.txt
2014_Qualitätsberichte.zip  2017_Qualitätsberichte.zip	locations.json
2015_Qualitätsberichte.zip  adresses2019.json


In [0]:
from zipfile import ZipFile
import re
from lxml import etree
from lxml import objectify
import pandas as pd


regex=re.compile('.*\/[0-9]{9}-99')
df_kkh=pd.DataFrame(columns=('IK', 'Standortnummer', 'Name', 'Jahr', 'Strasse', 'Hausnummer', 'Postleitzahl', 'Ort'))
df_icd=pd.DataFrame(columns=('IK', 'Standortnummer', 'Jahr', 'ICD_10', 'Fallzahl'))
store = pd.HDFStore('Qualitätsberichte.h5')

zipfilenames=['2013_Qualitätsberichte.zip', '2014_Qualitätsberichte.zip', '2015_Qualitätsberichte.zip', '2016_Qualitätsberichte.zip', '2017_Qualitätsberichte.zip']
for zipfilename in zipfilenames:
  with ZipFile(zipfilename) as zippy:
    for filename in zippy.namelist():
      if not(filename.lower().endswith('xml')) or regex.match(filename):
        continue
      try:
        jahr=zipfilename[0:4]
        xml_file=zippy.open(filename, 'r')
        root=etree.fromstring(xml_file.read())
        standort=root.xpath('/Qualitaetsbericht/Standort_dieses_Berichts/Kontaktdaten|/Qualitaetsbericht/Krankenhaus/Standortkontaktdaten')
        if len(standort)==0:
          standort=root.xpath('/Qualitaetsbericht/Krankenhaus/Krankenhauskontaktdaten|/Qualitaetsbericht/Krankenhaus/Kontaktdaten')
        obj_kkh=objectify.XML(etree.tostring(standort[0]))
        obj_adresse=obj_kkh.find('Hausanschrift')
        if obj_adresse==None:
          obj_adresse=obj_kkh.Kontakt_Zugang
        if obj_adresse.find('Hausnummer')==None: objectify.SubElement(obj_adresse, 'Hausnummer')
        data={'IK': obj_kkh.IK.text, 'Standortnummer': obj_kkh.Standortnummer.text, 'Name': obj_kkh.Name.text, 'Jahr': jahr, 'Strasse': obj_adresse.Strasse.text, 'Hausnummer': obj_adresse.Hausnummer.text, 'Postleitzahl': obj_adresse.Postleitzahl.text, 'Ort': obj_adresse.Ort.text}
        row=pd_kkh.Series(data)
        row.name=obj_kkh.IK.text+'_'+obj_kkh.Standortnummer.text+'_'+str(jahr)
        df_kkh.append(row)
        obj_fa=objectify.XML(etree.tostring(root.xpath('/Qualitaetsbericht/Organisationseinheiten_Fachabteilungen/Organisationseinheit_Fachabteilung')[0]))
        if obj_fa.find('Hauptdiagnosen/Hauptdiagnose')!=None:
          for i in range(0, len(obj_fa.Hauptdiagnosen.Hauptdiagnose)):
            obj=obj_fa.Hauptdiagnosen.Hauptdiagnose[i]
            fallzahl=0
            if obj.find('Fallzahl')!=None:
              fallzahl=obj['Fallzahl'].text
            data={'IK': obj_kkh.IK.text, 'Standortnummer': obj_kkh.Standortnummer.text, 'Jahr': jahr, 'ICD_10': obj['ICD_10'].text, 'Fallzahl': fallzahl}
            row=pd_icd.Series(data)
            row.name=obj_kkh.IK.text+'_'+obj_kkh.Standortnummer.text+'_'+str(jahr)+'_'+str(i)
            df_icd.append(row)
      except Exception as e:
        pass
store['df_kkh'] = df_kkh
store['df_icd'] = df_icd



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['IK', 'Standortnummer', 'Jahr', 'ICD_10', 'Fallzahl']]

  exec(code_obj, self.user_global_ns, self.user_ns)
